# Numerical Variables Analysis

We will be examining the categorical variables of the `kaggle_steam` dataset (available in `data/01_raw/kaggle_steam.csv`) in this notebook.

After analysis, we will propose categorical features to adopt for our clustering model, KPrototypes.

The dataset has **11** categorical variables: `appid`, `name`, `release_date`, `english`, `developer`. `publisher`, `platforms`, `categories`, `genres`, `steamspy_tags` and `owners`.

We will be examining the following categorical variables to see if we can use them for clustering.

- `release_date`
- `english`
- `platforms`
- `categories`
- `genres`
- `steamspy_tags`
- `owners`

We omitted:

- `app_id` as it is id value, it has no meaning,
- `name` is unique per record, hence has no meaning
- `developer` is immediately apparent to have many labels, it will increase dimensionality of our data too much and impact our model's performance
- `publisher` same as above

**Setting up**

In [1]:
%load_ext kedro.ipython
%load_ext autoreload
%matplotlib inline
%autoreload 2

[04/16/23 08:42:22] INFO     Resolved project path as: /home/eckt/Documents/usg.                    ]8;id=313731;file:///home/eckt/.local/share/virtualenvs/usg-8_lb1q9f/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=924904;file:///home/eckt/.local/share/virtualenvs/usg-8_lb1q9f/lib/python3.8/site-packages/kedro/ipython/__init__.py#134\134]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[04/16/23 08:42:25] INFO     Kedro project usg                                                      ]8;id=243156;file:///home/eckt/.local/share/virtualenvs/usg-8_lb1q9f/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=503073;file:///home/eckt/.local/share/virtualenvs/usg-8_lb1q9f/lib/python3.8/site-packages/kedro/ipython/__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=363400;file:///home/eckt/.local/share/virtualenvs/usg-8_lb1q9f/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=807142;file:///home/eckt/.local/share/virtualenvs/usg-8_lb1q9f/lib/python3.8/site-packages/kedro/ipython/__init__.py#104\104]8;;\
                             'pipelines'                                                                           

[04/16/23 08:42:27] INFO     Registered line magic 'run_viz'                                        ]8;id=254615;file:///home/eckt/.local/share/virtualenvs/usg-8_lb1q9f/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=291869;file:///home/eckt/.local/share/virtualenvs/usg-8_lb1q9f/lib/python3.8/site-packages/kedro/ipython/__init__.py#110\110]8;;\

In [2]:
import pandas as pd
import polars as pl
import numpy as np

from matplotlib import rc_context
import matplotlib.pyplot as plt
import seaborn as sb
from seaborn.objects import Plot
import seaborn.objects as so

from sklearn.preprocessing import maxabs_scale

import logging

from usg.utils import *

log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
sb.set()

**All features of `kaggle_steam` and their dtypes**

The dataset has **11** categorical variables: `appid`, `name`, `release_date`, `english`, `developer`. `publisher`, `platforms`, `categories`, `genres`, `steamspy_tags` and `owners`.

In [3]:
kaggle_steam: pd.DataFrame = catalog.load('kaggle_steam')
kaggle_steam.dtypes.to_frame().T

[04/16/23 08:43:11] INFO     Loading data from 'kaggle_steam' (CSVDataSet)...                   ]8;id=296584;file:///home/eckt/.local/share/virtualenvs/usg-8_lb1q9f/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=360701;file:///home/eckt/.local/share/virtualenvs/usg-8_lb1q9f/lib/python3.8/site-packages/kedro/io/data_catalog.py#343\343]8;;\

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,int64,object,object,int64,object,object,object,int64,object,object,object,int64,int64,int64,int64,int64,object,float64


# Summary

In [4]:
ratings_ratio = kaggle_steam['positive_ratings'] / kaggle_steam['negative_ratings']
ratings_ratio.loc[np.isinf(ratings_ratio)] = 0

In [7]:
keep = pd.concat([
    kaggle_steam[['appid', 'required_age', 'achievements', 'positive_ratings', 'negative_ratings', 'average_playtime', 'median_playtime', 'price']], 
    ratings_ratio.to_frame('ratings_ratio'), 
], axis=1)

catalog.save('features_eng_2', keep)

[04/16/23 08:51:29] INFO     Saving data to 'features_eng_2' (CSVDataSet)...                    ]8;id=321305;file:///home/eckt/.local/share/virtualenvs/usg-8_lb1q9f/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=185264;file:///home/eckt/.local/share/virtualenvs/usg-8_lb1q9f/lib/python3.8/site-packages/kedro/io/data_catalog.py#382\382]8;;\